In [3]:
import pandas as pd

Load reservation data and merge columns.

In [17]:
df = pd.read_csv("SELECT_overnattingsdato__koienavn__SUM_a.csv", header=None)
df.columns=["DS", "Koie", "M", "NM"]
df.head()

,DS,Koie,M,NM
0,1999-10-08,Agdenes,2,0
1,1999-10-09,Agdenes,5,2
2,1999-10-15,Agdenes,6,1
3,1999-10-16,Agdenes,6,1
4,1999-10-22,Agdenes,5,5


In [18]:
df["Y"] = df["M"] + df["NM"]
df = df.drop(["M", "NM"], axis=1)
df.head()

,DS,Koie,Y
0,1999-10-08,Agdenes,2
1,1999-10-09,Agdenes,7
2,1999-10-15,Agdenes,7
3,1999-10-16,Agdenes,7
4,1999-10-22,Agdenes,10


Group the dataframe by cabin name

In [26]:
gdf = df.groupby("Koie")
gdf